# Identity - List access packages

<table align="left">
    <tr>
        <th>Permission type</th>
        <th>Permissions (from least to most privileged)</th>
    <tr align="center">
        <td>Delegated (work or school account)</td>
        <td>EntitlementManagement.Read.All, EntitlementManagement.ReadWrite.All</td>
    </tr>
    <tr align="center">
        <td>Delegated (personal Microsoft Account)</td>
        <td>Not Supported</td>
    </tr>
    <tr align="center">
        <td>Application</td>
        <td>Not Supported</td>
</table>

## Initialize variables
Required Variables

In [17]:
$config = @{
    tenantDomain = "domain.com"
    tenantID = "xxxxxxx-xxxx-xxxx-xxxxx-xxxxxxxxxx"          ## Azure AD Tenant ID
    appID = "xxxxxx-23af-4ae5-a6ed-xxxxxx"                   ## Usually known as Client ID or Application ID            ## This is the secretKey. Important only authorized users have this 
    resourceURI = "https://graph.microsoft.com"   ## This is the resource endpoint for MDATP
}

## Import variables

In [ ]:
$credentialPath = ".\graph.credential"

if (!$config) {  ## Checks to see if config file exist and if not export credential file from above.
    Try {
        $config = (Import-CliXml -Path $credentialPath)
        Write-Host -Foreground Green "Imported credentials from $credentialPath"
    } catch {
        Write-Host -foreground yellow "`nNo Credential found or in memory or stored in $credentialPath.`nPlease provide credentials in cell (1) or specify path in $credentialpath"
    }
} else {
    Write-host "New Credentials provided in Cell (1) instead of importing credentials. `nStore credentials in $credentialPath (Y or N)" -ForegroundColor Yellow 
        $Readhost = Read-Host " ( Y / N ) " 
        Switch ($ReadHost) 
         { 
           Y {Write-host "Storing credentials"; $config | Export-CliXml -Path $storagePath} 
           N {Write-Host "Okay, I wont store the credentials"; $PublishSettings=$false} 
           Default {Write-Host "No action taken"; $PublishSettings=$false} 
         } 
}

## Check Variables

In [ ]:
$config.authUrl = "https://login.microsoftonline.com/$($config.tenantDomain)";
Write-Host -ForeGroundColor Yellow "`nAuthentication Parameters that will be used"
Write-Host "Resource URL: $($config.resourceURI)"
Write-Host "Client APP ID: $($config.appID)"
Write-Host "AuthUrl: $($config.authUrl)"

## Import Graph Authentication module to authenticate by Device Code
##### We will be importing the Device Code Authentication Module. To learn about this module go to notebook [here](https://demoloaderpowershellnotebook-skyserver.msappproxy.net/lab/tree/Graph-API/Authentication%20-%20Rest%20API%20-%20Device%20Code.ipynb)

In [ ]:
Import-Module ".\modules\GraphAuthentication.psm1" -Force  ## Imports the module
#Get-Command -Module "GraphAuthentication"                  ## Shows what commands were imported

$tokenResponse = Invoke-GraphAuthenticateDeviceCode -clientAppId $config.appID -tenantDomain $config.tenantDomain

## Make your Request Parameters for Graph Rest Call

In [ ]:
## Request Parameters
$URI = "https://graph.microsoft.com/beta/identityGovernance/entitlementManagement/accessPackages" ## URL of what resource to query on Graph API
$authHeader = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" 
}

Write-Host "`nURI:",$URI
Write-Host "AUth Header:", ($authHeader | ConvertTo-JSON)

## Call Graph API for list of access packages
Calling the url ```https://graph.microsoft.com/beta/identityGovernance/entitlementManagement/accessPackages```

In [ ]:
$Result = (Invoke-RestMethod -Method Get -Uri $URI -Headers $authHeader -ErrorAction Stop)

Write-Host -ForeGroundColor Cyan "`nHere is an example of one retrieved access package"
$Result.value | Select-Object -First 1 | FL  ## This will take the results and limit it to 1 item.

## Parse Results

In [ ]:
#query Graph API and insert into $graphresponse variable
$filterObjects = @('displayName','id','catalogId','description')
$Result = (Invoke-RestMethod -Method Get -Uri $URI -Headers $authHeader -ErrorAction Stop)
$Result.value | Select-Object -Property $filterObjects

In [39]:
$endpoint = "$($config.resourceURI)/beta/identityGovernance/entitlementManagement/accessPackages/e1c1bc0a-bf98-4370-97bb-752f00bac575"; $endPoint
$Response = (Invoke-RestMethod -Method Get -Uri $endpoint -Headers $authHeader -ErrorAction Stop)

https://graph.microsoft.com/beta/identityGovernance/entitlementManagement/accessPackages/e1c1bc0a-bf98-4370-97bb-752f00bac575


In [ ]:
$Response